# Load Data

In [46]:
import os
import pandas as pd
folder_path = 'Data_Test_v1'
file_names = ['data_test.csv', 'data_train.csv', 'target_test.csv', 'target_train.csv']

data_frames = []
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path)
    data_frames.append(df)

data_test = data_frames[0]
data_train = data_frames[1]
target_test = data_frames[2]    
target_train = data_frames[3]

In [47]:
# Initialize an empty list to store the final order quantities
final_order_quantities_ANN = []
final_order_quantities_DT = []

# Define costs
c = 2  # cost per unit of product
s = 1  # salvage value per unit of leftover product
p = 3  # price per unit of product sold

# Calculate critical ratio
critical_ratio = (p - c) / ((p - s) + (p - c))

# ANN

In [48]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np

# Define a custom loss function for the newsvendor problem with quantile regression
def newsvendor_quantile_loss(q, y_true, y_pred):
    error = y_true - y_pred
    return tf.keras.backend.mean(tf.maximum(q*error, (q-1)*error), axis=-1)

In [49]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.optimizers import Adam

def create_model(n_hidden=1, n_neurons=30, input_shape=[3]): #learning_rate_input=3e-3, 
    model = Sequential()
    model.add(Dense(n_neurons, activation="relu", input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(Dense(n_neurons, activation="relu"))
    model.add(Dense(1))
    optimizer = Adam() #learning_rate=learning_rate_input
    model.compile(loss=lambda y,f: newsvendor_quantile_loss(critical_ratio,y,f), optimizer=optimizer)
    return model

In [50]:
def model_builder(n_hidden=1, n_neurons=30, ): #learning_rate=3e-3
    return KerasRegressor(build_fn=create_model, verbose=0, n_hidden=n_hidden, n_neurons=n_neurons) #, learning_rate=learning_rate

In [51]:
model_ANN = model_builder()

from scipy.stats import reciprocal

param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 100),
}

rnd_search_cv_ANN = RandomizedSearchCV(model_ANN, param_distribs, n_iter=10, cv=3)

target_train = target_train.astype('float32')

rnd_search_cv_ANN.fit(data_train, target_train)

c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future relea

KeyboardInterrupt: 

In [ ]:
target_pred_ANN = rnd_search_cv_ANN.predict(data_test)
print(target_pred_ANN)   

[[572.19965]
 [571.7527 ]
 [571.48444]
 [571.7565 ]
 [572.2775 ]
 [570.68085]
 [571.5103 ]
 [572.45404]
 [571.4966 ]
 [571.7686 ]
 [570.70514]
 [571.4361 ]
 [571.41187]
 [571.6906 ]
 [570.3764 ]
 [571.4982 ]
 [570.87317]
 [569.403  ]
 [570.6066 ]
 [569.97797]
 [572.49567]
 [570.487  ]
 [571.0619 ]
 [572.03516]
 [571.4929 ]
 [570.2203 ]
 [570.42645]
 [570.7172 ]
 [571.20575]
 [570.184  ]
 [570.4385 ]
 [570.3642 ]
 [570.77094]
 [571.55035]
 [571.7564 ]
 [572.1874 ]
 [571.58826]
 [571.0414 ]
 [570.4627 ]
 [571.73224]
 [571.5262 ]
 [571.2838 ]
 [571.0672 ]
 [571.19366]
 [570.8073 ]
 [570.52856]
 [571.29596]
 [571.3338 ]
 [572.1337 ]
 [571.48987]
 [571.5588 ]
 [570.66876]
 [570.5165 ]]


# DT

In [ ]:
import numpy as np
import lightgbm as lgb

In [ ]:
# Specify configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'quantile',
    'alpha': critical_ratio,
    'metric': 'quantile',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

# Create a dataset for LightGBM
lgb_train = lgb.Dataset(data_train, label=target_train)

# Train the model
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20)

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [ ]:
# Predict the demand
target_pred_DT = gbm.predict(data_test, num_iteration=gbm.best_iteration)
print(target_pred_DT)

[1588.51045508 1347.071103   1351.43863981 1574.24690374 1673.14954207
 1394.87521291 1564.57032572 1209.67883273 1351.43863981 1659.13083472
 1401.14380057 1574.24690374 1392.20648385 1201.01498805 1320.83385063
 1347.071103   1515.04397316 1310.10892914 1463.50411418 1310.10892914
 1403.67813752 1295.98826923 1465.50363782 1644.38683456 1193.49232968
 1343.72279908 1394.87521291 1401.19443837 1630.12328322 1401.14380057
 1401.14380057 1320.83385063 1562.56381333 1592.27328927 1632.20544916
 1666.88095441 1557.95084044 1364.76974715 1401.14380057 1392.20648385
 1630.12328322 1534.43985721 1618.47920402 1658.88599073 1465.50363782
 1533.01399285 1365.01459114 1613.04010694 1193.49232968 1652.61740307
 1193.49232968 1276.91333478 1562.56381333]


# Costs

In [52]:
# Loop over each week in target_test
overall_profit_ANN = 0
overall_profit_DT = 0
for i in range(len(target_test)):

    # Calculate the profit for the week (without over or understock costs)
    profit_ANN = (p - c) * np.round(target_pred_ANN[i])
    profit_DT = (p - c) * np.round(target_pred_DT[i])

    # Calculate understock and overstock costs
    overstock_ANN = 0
    overstock_DT = 0
    understock_ANN = 0
    understock_DT = 0
    if target_pred_ANN[i] < target_test.values[i]:
        understock_ANN = (p - c) * (target_test.values[i] - np.round(target_pred_ANN[i]))
    if target_pred_ANN[i] > target_test.values[i]:
        overstock_ANN = (c - s) * (np.round(target_pred_ANN[i]) - target_test.values[i])
    if target_pred_DT[i] < target_test.values[i]:
        understock_DT = (p - c) * (target_test.values[i] - np.round(target_pred_DT[i]))
    if target_pred_DT[i] > target_test.values[i]:
        overstock_DT = (c - s) * (np.round(target_pred_DT[i]) - target_test.values[i])
    

    # Calculate the total profit for the week
    total_profit_ANN = profit_ANN - overstock_ANN - understock_ANN
    total_profit_DT = profit_DT - overstock_DT - understock_DT

    # Add the total profit to the overall profit
    overall_profit_ANN += total_profit_ANN
    overall_profit_DT += total_profit_DT
   

# Print the overall profit
print('Overall profit for ANN: ', int(overall_profit_ANN))
print('Overstock: ', int(overstock_ANN), ' Understock: ', int(understock_ANN))
print('Overall profit for DT: ', int(overall_profit_DT))
print('Overstock: ', int(overstock_DT), ' Understock: ', int(understock_DT))

Overall profit for ANN:  -21451
Overstock:  0  Understock:  1280
Overall profit for DT:  65847
Overstock:  0  Understock:  288


C:\Users\lanza\AppData\Local\Temp\ipykernel_7724\4077709580.py:35: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('Overall profit for ANN: ', int(overall_profit_ANN))
C:\Users\lanza\AppData\Local\Temp\ipykernel_7724\4077709580.py:36: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('Overstock: ', int(overstock_ANN), ' Understock: ', int(understock_ANN))
C:\Users\lanza\AppData\Local\Temp\ipykernel_7724\4077709580.py:37: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1